In [11]:
from __future__ import print_function
from PIL import Image, ImageChops, ImageEnhance
import sys, os
import glob
import time
import joblib
from joblib import Parallel, delayed

In [3]:
TMP_EXT = "_tmp_ela.jpg"
ELA_EXT = "_ela.jpg"


In [4]:
def convert_to_ela_image(path, save_dir, quality):
    filename = path
    file = filename.split("/")[-1].split(".")[0]
#     print(file)
#     file = file.split(".")[0]
#     print(file)
    basename, ext = os.path.splitext(filename)
    tmp_fname = os.path.join(save_dir, file + TMP_EXT)
    ela_fname = os.path.join(save_dir, file + ELA_EXT)
#     print(basename)
#     print(ext)
#     print(ela_fname)

    # resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    # ELA_filename = filename.split('.')[0] + '.ela.png'
    try:
        original = Image.open(filename).convert('RGB')
        original.save(tmp_fname, 'JPEG', quality=quality)
        resaved_original = Image.open(tmp_fname)

        ela_im = ImageChops.difference(original, resaved_original)

        extrema = ela_im.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0:
            max_diff = 1
        scale = 255.0 / max_diff

        ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)

        ela_im.save(ela_fname)
        os.remove(tmp_fname)
    except:
        print("corrupted image file")
        pass

In [13]:
img_dir_real = "/home/FRACTAL/shaheen.perveen/stage3/fakers/stage-3/stage3_image_data/train/real/"
data_path = os.path.join(img_dir_real,'*.jpg') 
save_dir = "/home/FRACTAL/shaheen.perveen/stage3/fakers/stage-3/stage3_image_data/train_ela/real/"

files = glob.glob(data_path)
# # loop format
# # real images
# for fname in files:
#     print(fname)
#     convert_to_ela_image(fname, save_dir, 95)

In [14]:
# parallel format real images
start = time.time()
Parallel(n_jobs=-1)(delayed(convert_to_ela_image)(fname,save_dir,95) for fname in files)
end = time.time()
print(end-start)

132.57782220840454


In [17]:
# fake images
img_dir_fake = "/home/FRACTAL/shaheen.perveen/stage3/fakers/stage-3/stage3_image_data/train/fake/"
data_path = os.path.join(img_dir_fake,'*.jpg') 
save_dir = "/home/FRACTAL/shaheen.perveen/stage3/fakers/stage-3/stage3_image_data/train_ela/fake/"

files = glob.glob(data_path)

In [18]:
# fake images in parallel
start = time.time()
Parallel(n_jobs=-1)(delayed(convert_to_ela_image)(fname,save_dir,95) for fname in files)
end = time.time()
print(end-start)

215.61451125144958


In [18]:
# solving the problem due to corrupted images
img_dir_fake = "/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/"
data_path = os.path.join(img_dir_fake,'*.jpg') 
save_dir = "/home/ubuntu/fakers/stage-3/stage3_image_data/train_ela/fake/"

files_all = glob.glob(data_path)
# type(files)

files_read = glob.glob(os.path.join(save_dir,'*.jpg'))
print(len(files_all))
print(len(files_read))

23887
22244


In [30]:
# extracting fiel name sot find out which images are corrputed
files_all = glob.glob(data_path)
files_all = [x.split("/")[-1].split(".")[0] for x in files_all]
# type(files)

files_read = glob.glob(os.path.join(save_dir,'*.jpg'))
files_read = [x.split("/")[-1].split(".")[0] for x in files_read]
files_read = list(map(lambda x: x[:-4],files_read))
# files_read = files_read.apply(lambda x:x[:-4],files_read)
print(len(files_all))
print(len(files_read))

23887
22244


In [36]:
# files not read due to corrupted images
files_notRead = [x for x in files_all if x not in files_read]
print(len(files_notRead))

1646


In [37]:
files_notRead[:5]

['new_fake20190208093216_new_fake20190208091910_f5de5c6f3d7299f33ac2a855afe0bd3d',
 'new_fake20190208093505_new_fake20190208091119_4895d66f174bfb1e91dec6dec44c3acc',
 'new_fake20190208094007_new_fake20190208091838_3d8cc220f4a11365f006dd878fb5c706',
 'new_fake20190208093509_new_fake20190208091159_ebf6fef38bf42bcd316a8014ab1ebc63',
 'new_fake20190208094304_e0c905387ebbdbe5abd4188b89709ee6']

In [38]:
# appending file location to file list which were not read
img_dir_fake = "/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/"
ext = '.jpg'
files_notRead = [img_dir_fake + x + ext for x in files_notRead]
print(files_notRead[:3])

['/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093216_new_fake20190208091910_f5de5c6f3d7299f33ac2a855afe0bd3d.jpg', '/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093505_new_fake20190208091119_4895d66f174bfb1e91dec6dec44c3acc.jpg', '/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094007_new_fake20190208091838_3d8cc220f4a11365f006dd878fb5c706.jpg']


In [42]:
len(files_notRead)

1646

In [43]:
# reading and converting images left out into fake_remaining directory
img_dir_fake = "/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/"
save_dir = "/home/ubuntu/fakers/stage-3/stage3_image_data/train_ela/fake_remaining/"

for fname in files_notRead:
    print(fname)
    convert_to_ela_image(fname, save_dir, 95)

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093216_new_fake20190208091910_f5de5c6f3d7299f33ac2a855afe0bd3d.jpg
corrupted image file
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093505_new_fake20190208091119_4895d66f174bfb1e91dec6dec44c3acc.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094007_new_fake20190208091838_3d8cc220f4a11365f006dd878fb5c706.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093509_new_fake20190208091159_ebf6fef38bf42bcd316a8014ab1ebc63.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094304_e0c905387ebbdbe5abd4188b89709ee6.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093924_new_fake20190208091354_5888bc2cecf88a6cd764b36b262304f3.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094107_new_fake20190208091845_5038efcaed79fda84729a16b80bff740.jpg
/home/ubuntu/fakers/st

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094224_4fe95de5df167016c7d53780a1fd0b30.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093534_new_fake20190208091523_0c46aa41e63fd95a3660fa57f1952068.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093536_new_fake20190208091929_bac5c555cdd83c8d13e8109fe3d916e4.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093719_new_fake20190208091750_d3ed471953e5895825910790106d9dcc.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093517_new_fake20190208091427_f81a162cdd4b93bed6a3da0dc7760d44.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093714_new_fake20190208091155_4a622e1ca1ecb2f4654c578eacb74d41.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093648_new_fake20190208091934_4b94632ff888eadab5bfa1580db97698.jpg
/home/ubuntu/fakers/stage-3/stage3_image_da

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094028_new_fake20190208091031_eea4d89eb7c9bd6794a5568c2d0b2e53.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093933_new_fake20190208091702_01f0ca81700fad9fd81a417f56f40912.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093819_new_fake20190208091531_5975e7c8a5e3436b1ae65758fb9ec885.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094125_new_fake20190208091722_8245d75fd04ec0dd49890cb211d422ee.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093637_new_fake20190208091025_c05addc07e4c8ab4b3bbfd1a77906799.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093749_new_fake20190208091149_435e92365eb1828873db5cbaa9a0cc7f.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094147_new_fake20190208091850_e58a48144f3c5d7869e0382e03805d06.jpg
/home/ubuntu/fakers/

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093710_new_fake20190208091319_3a570ef4e9dd78964611a8e2a798bcdb.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094022_new_fake20190208091129_cd77533a59c5ee63afdedcbf06fdfbc5.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093543_new_fake20190208091047_13a51ca25b3d3c5c8a06bdccf9b327ff.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094045_new_fake20190208091103_85b82c2c2c32152f5250bfa4dce76635.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093901_new_fake20190208091217_af5087b39e9eb8d49d3fd01305c1895b.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094203_new_fake20190208091202_53555cee486dd0d89a9ed8b87cdc3c2d.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094203_new_fake20190208091035_6d17412e5b163b9500dc49324ce389c6.jpg
/home/ubuntu/fakers/

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094049_new_fake20190208091455_0073ee498a270024c2bf00eddfa941cd.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093354_new_fake20190208091300_3d647c6753ca9ae036aae502ef68d2d6.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093954_new_fake20190208091136_d267fa171bb35fb786fc726da5515ba9.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094214_new_fake20190208091406_e10d6f8a9e562c2e222d2ae498c9af8a.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093427_new_fake20190208091049_971ac01fe28cb6d7d429b4c9de4b3d14.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093544_new_fake20190208091038_c3192d938132692ddd299834296c4cb4.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093217_new_fake20190208091806_5b22303a72cb08d61ad033058ecc3f23.jpg
/home/ubuntu/fakers/

...

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094024_new_fake20190208091746_434220a2086b0e519cb9ba3045f55f7d.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093814_new_fake20190208091453_e9e94321622cd1c446338ca989c36374.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093504_new_fake20190208091549_3db4caef2dd862b20d8e997c4a6bac9a.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094158_new_fake20190208091442_83f221c69c4b8f1faea7f790bad9dc86.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093501_new_fake20190208091250_049081279c946118b377185382fc1a93.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094116_new_fake20190208091121_6e51bd4c4e4207ba2de733c2fb565337.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093553_new_fake20190208091734_9eaf268e185ae56aa74289a83435a4e3.jpg
/home/ubuntu/fakers/

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093844_new_fake20190208091919_91255395407809bd96f027bf99c73820.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093206_new_fake20190208091910_1474ad08ec6438cfda08faa3786a963e.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093419_new_fake20190208091447_cfae1b652ffba0812a434c9f735cedd9.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093443_new_fake20190208091104_b3a66679ce9a19b9d6972b7dd1dd781d.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094157_new_fake20190208091347_58df253b8e0f6eae9703d1347ac33947.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093942_new_fake20190208091422_bba27bd0e4864f1aeea1623accd4b32c.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094018_new_fake20190208091419_372ef0c0a9833272ccd8fed9c636c51e.jpg
/home/ubuntu/fakers/

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094104_new_fake20190208091845_f5d8154a6844989ee460f7738e4d1b07.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094041_new_fake20190208091233_5532e788a590adb3d79e52a531329c8f.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094149_new_fake20190208091850_27da91a47a4f9d693699053c5a13fd2e.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093316_new_fake20190208091506_3e0230374e4eccf3d5f8c12d03b517ca.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093724_new_fake20190208091857_62ac6098883ecfaa489d7e974dc32c68.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093351_new_fake20190208091502_9361f150a7180a2fa43c64fed9b189cb.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093459_new_fake20190208091351_b5a884055bc4bec19d2f314627e18160.jpg
/home/ubuntu/fakers/

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093505_new_fake20190208091119_54b5bbae012dce5fc62abec67b913559.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093200_new_fake20190208091117_64ede1b4f6796bf430e28d7d9ceaff7f.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094111_new_fake20190208091524_865205bac868cc2d04da42a66e35b876.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093802_new_fake20190208091408_fd1c0fa82534c516f8662c7f8b4f3ba7.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093806_new_fake20190208091634_6f5b837553b280010ae30f0b6d3bd003.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094139_new_fake20190208091649_39a6d173ef11da2ce589b63eebf16fce.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094138_new_fake20190208091649_055dbd33feb1274bbe1919a827d3b153.jpg
/home/ubuntu/fakers/

/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094048_new_fake20190208091542_7ad6c4b9708b146dd6de0d48b61db1c8.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093502_new_fake20190208091223_cda2367a0343459f0f75272e07dd70fb.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093542_new_fake20190208091550_c4f269522f76d019a7aadec7fc2911dc.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093903_new_fake20190208091933_e5496d80dc69933df4e10bb0f3ee5acf.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208094202_new_fake20190208091044_cb3c8f0296976de8db2759fc008f019b.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093411_new_fake20190208091552_d4d9d249eed9bdd918fbb91028257911.jpg
/home/ubuntu/fakers/stage-3/stage3_image_data/train/fake/new_fake20190208093337_new_fake20190208091637_2e815fd795ba15316b141cdca6d6f14a.jpg
/home/ubuntu/fakers/

In [44]:
len(files_notRead)

1646

In [49]:
files_remain = list(glob.glob(os.path.join(save_dir, '*')))

In [50]:
len(files_remain)

1645